In [3]:
import pandas as pd
import numpy as np

df_WS = pd.read_excel('WS.xlsx')  # Dataset containing WS values
# Load the dataset
df_total = pd.read_excel("total_dataset_for_FOM.xlsx")

# Clean up column names by stripping spaces
df_total.columns = df_total.columns.str.strip()

# Define unique combinations of parameters without dropping samples
unique_combinations = df_total[['d1(um)', 'd2(um)', 'p(um)', 'tg(um)', 'na']].drop_duplicates()

# Store results
results = []

# Iterate through each unique combination
for _, row in unique_combinations.iterrows():
    condition = (
        (df_total['d1(um)'] == row['d1(um)']) &
        (df_total['d2(um)'] == row['d2(um)']) &
        (df_total['p(um)'] == row['p(um)']) &
        (df_total['tg(um)'] == row['tg(um)']) &
        (df_total['na'] == row['na'])
    )

    # Filter the dataset based on the condition
    filtered_df = df_total[condition].reset_index(drop=True)

    if not filtered_df.empty:
        # Calculate the maximum confinement loss
        max_CL = filtered_df['Confinement Loss'].max()
        HM = max_CL / 2  # Calculate the half-maximum value

        # Find the index where the value is maximum
        max_index = filtered_df['Confinement Loss'].idxmax()
        
        max_CL_wl= filtered_df.loc[max_index, 'wl']
        
        # Boundary check to avoid index errors
        if max_index == 0 or max_index >= len(filtered_df) - 1:
            continue
 
        # Find left side closest points to HM
        left_df = filtered_df.loc[:max_index - 1]
        left_closest_idx = left_df['Confinement Loss'].sub(HM).abs().idxmin()

        # Ensure index does not exceed bounds
        if left_closest_idx + 1 >= len(filtered_df):
            continue
        
        x1_left, x2_left = filtered_df.loc[left_closest_idx, 'wl'], filtered_df.loc[left_closest_idx + 1, 'wl']
        y1_left, y2_left = filtered_df.loc[left_closest_idx, 'Confinement Loss'], filtered_df.loc[left_closest_idx + 1, 'Confinement Loss']

        # Linear interpolation for x_left
        x_left = x1_left + ((HM - y1_left) / (y2_left - y1_left)) * (x2_left - x1_left)

        # Find right side closest points to HM
        right_df = filtered_df.loc[max_index + 1:]
        right_closest_idx = right_df['Confinement Loss'].sub(HM).abs().idxmin()
            
        # Ensure index does not exceed bounds
        if right_closest_idx + 1 >= len(filtered_df):
            continue

        x1_right, x2_right = filtered_df.loc[right_closest_idx-1, 'wl'], filtered_df.loc[right_closest_idx, 'wl']
        y1_right, y2_right = filtered_df.loc[right_closest_idx-1, 'Confinement Loss'], filtered_df.loc[right_closest_idx, 'Confinement Loss']

        # Linear interpolation for x_right
        x_right = x1_right + ((HM - y1_right) / (y2_right - y1_right)) * (x2_right - x1_right)

        # Calculate Full Width at Half Maximum (FWHM)
        FWHM = x_right - x_left

        
 # Find corresponding WS value from WS.csv using matching conditions
        WS_row = df_WS[
            (df_WS['d1(um)'] == row['d1(um)']) &
            (df_WS['d2(um)'] == row['d2(um)']) &
            (df_WS['p(um)'] == row['p(um)']) &
            (df_WS['tg(um)'] == row['tg(um)']) &
            (df_WS['na'] == row['na'])
        ]

        if not WS_row.empty:
            WS = WS_row.iloc[0]['WS']
            # Calculate Figure of Merit (FOM)
            FOM = WS / (FWHM * 1000)  # Convert μm to nm
        else:
            WS = None
            FOM = None
        
        # Store results
        results.append([
            row['d1(um)'], row['d2(um)'], row['p(um)'], row['tg(um)'], row['na'], max_CL_wl, max_CL, HM,
            x1_left, x2_left, y1_left, y2_left,
            x1_right, x2_right, y1_right, y2_right, x_left, x_right, 
            FWHM, WS, FOM
        ])

# Convert results to a DataFrame
columns = [
    'd1(um)', 'd2(um)', 'p(um)', 'tg(um)', 'na', 'max_wl', 'max_CL', 'HM',
    'x1_left', 'x2_left', 'y1_left', 'y2_left',
    'x1_right', 'x2_right', 'y1_right', 'y2_right', 'x_left', 'x_right',
    'FWHM', 'WS', 'FOM'
]

df_results = pd.DataFrame(results, columns=columns)

# Save the results to a CSV file

df_results.to_excel("FOM.xlsx", index=False)
# Display the results
print(df_results)



     d1(um)  d2(um)  p(um)  tg(um)    na  max_wl     max_CL         HM  \
0       0.6     1.0    2.0    0.03  1.31    0.55  57.787134  28.893567   
1       0.6     1.0    2.0    0.04  1.31    0.57  50.386646  25.193323   
2       0.6     1.0    2.0    0.05  1.31    0.58  29.812057  14.906029   
3       0.6     1.0    2.0    0.06  1.31    0.59  17.478406   8.739203   
4       1.6     0.8    2.0    0.04  1.31    0.57  71.040651  35.520325   
..      ...     ...    ...     ...   ...     ...        ...        ...   
167     0.6     1.0    3.0    0.06  1.37    0.70  12.357817   6.178908   
168     0.6     1.0    3.0    0.06  1.38    0.73  15.957427   7.978714   
169     0.6     1.0    3.0    0.06  1.39    0.78  20.098783  10.049391   
170     0.6     1.0    3.0    0.06  1.40    0.85  25.842559  12.921280   
171     0.6     1.0    3.0    0.06  1.41    0.96  34.638712  17.319356   

     x1_left  x2_left  ...    y2_left  x1_right  x2_right   y1_right  \
0       0.52     0.53  ...  40.745854  